In [28]:
import pandas as pd
import numpy as np

In [2]:
books = pd.read_csv("cleaned_books_dataset.csv")
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tag_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883_A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web_A Novel,9780002261982_A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736_A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897_Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934_""In The Problem of Pain, C.S. Le..."


In [3]:
books['categories'].value_counts().reset_index().query('count > 50')

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
5,Religion,117
6,Philosophy,117
7,Comics & Graphic Novels,116
8,Drama,86
9,Juvenile Nonfiction,57


In [4]:
pd.set_option("display.max_colwidth", None)

In [5]:
books[books['categories']=="Juvenile Fiction"][['title','categories', 'description']]

,title,categories,description
30,Ocean Star Express,Juvenile Fiction,"Joe and his parents are enjoying a summer holiday by the sea at the Ocean Star Hotel. The sky is bright blue, the sun shines and Joe loves all that the seaside has to offer. But when the fog rolls in and rain falls Joe begins to wish that he was back at home again. Things change, however, when the owner of the hotel invites Joe to share in a magical world, only a few steps away. The loft is black as night but then above Joe's head a thousand tiny stars begin to sparkle and in the distance he hears the chug-chug-chug of a model train. A whole world is soon to open up before Joe's eyes, a world of snow-capped mountains, great deserts, and rocking fishing boats."
79,The voyage of the Dawn Treader,Juvenile Fiction,"The ""Dawn Treader"" is the first ship Narnia has seen in centuries. King Caspian has built it for his voyage to find the seven lords, good men whom his evil uncle Mizaz banished when he usurped the throne. The journey takes Edmund, Lucy, and their cousin Eustace to the Eastern Islands, beyond the Silver Sea, toward Aslan's country at the End of the World. Illustrations."
85,Where the Red Fern Grows,Juvenile Fiction,A young boy living in the Ozarks achieves his heart's desire when he becomes the owner of two redbone hounds and teaches them to be champion hunters.
86,Poppy's Return,Juvenile Fiction,"There's trouble at Gray House, the girlhood home that Poppy left long ago. Poppy's family has called her back to save them all—mother, father, sisters and brothers, and dozens and dozens of deer mouse cousins. Poppy invites her rebellious son, Junior, to join her on the long trip across Dimwood Forest, hoping the journey will bring them closer together. But with Junior's skunk pal, Mephitis, and Ereth, the cantankerous porcupine, in tow—sugared slug soup!—Poppy and Junior may be in for unexpected adventure."
87,Diary of a Spider,Juvenile Fiction,"This is the diary ... of a spider. But don't be worried – he's more scared of you and your gigantic shoe! Actually, he's a lot like you. He goes to gym class and has Grandparents' Day at school. But he also spins sticky webs, scales walls, and takes wind–catching lessons. Lucky for him, his best friend is a fly! Doreen Cronin and Harry Bliss, the team behind the #1 bestselling Diary Of A Worm, spin a hilarious tale about the upside–down web world of an eight–legged charmer and his unlikely friend, Fly."
...,...,...,...
4845,Fablehaven,Juvenile Fiction,"When Kendra and Seth go to stay at their grandparents' estate, they discover that it is a sanctuary for magical creatures and that a battle between good and evil is looming."
4930,Sherlock Holmes and the Case of the Hound of the Baskervilles,Juvenile Fiction,Sherlock Holmes and Dr. Watson travel to the bleak wastes of Dartmoor to solve the mystery surrounding the late Sir Charles Baskerville and a ghostly hound.
4942,The Drift House,Juvenile Fiction,"Sent to stay with their uncle in a ship-like home called Drift House, twelve-year-old Susan and her stepbrothers embark on an adventure involving duplicitous mermaids, pirates, and an attempt to stop time forever."
5010,Attack of the Jaguar,Juvenile Fiction,"This training manual for operatives of Xtreme Adventures Inc., tells the story of Simon, a teenager who finds himself alone in the depths of the Amazon, where he must rescue his parents and survive the dangers of the jungle."


In [6]:
books[books['categories']=="Juvenile Nonfiction"][['title','categories', 'description']].head()

,title,categories,description
107,The Secret Garden Cookbook,Juvenile Nonfiction,"Frances Hodgson Burnett's The Secret Garden delights in the wondrous discoveries of lonely Mary Lennox as she slowly helps bring an abandoned garden back to life. It also delights in good food and a robust appetite, and the health and strength they can bring. Written at a time when many children were going hungry and even starving, Burnett's beloved story celebrates the magic of fresh air, new milk, homemade currant buns, and hearty, simple fare. Inside the pages of this cookbook are recipes for Mary's favorite foods, in and out of the garden, from porridge to roasted potatoes and eggs, all inspired by The Secret Garden and all adapted by chef and culinary historian Amy Coder from traditional Victorian recipes. Ms. Coder has supplemented these simple, wholesome recipes with fascinating tidbits on Victorian foods and Victorian eating customs. The result is a scrumptious tribute to Burnett's classic novel, a fascinating glimpse into the cooking customs of a historical period that is now long past, and a step-by-step guide to making delicious, tasty treats to enjoy in your own secret garden."
108,Laura's Album,Juvenile Nonfiction,"Though best known as the author of the Little House books, Laura Ingalls Wilder led a full, rich life that spanned almost a century of American history. All through her life Laura saved mementos of her past, including early writings, letters, drawings, and photographs, which have been lovingly preserved in private and public collections across the country. Now, for the first time ever, these photographs, writings, and memorabilia have been gathered together in one incredible volume by noted Little House historian William Anderson. Each gorgeous page of LAURA'S ALBUM is a doorway into the private world of Laura Ingalls Wilder, and offers a unique glimpse of what her life was like. Here is the fascinating true story of this remarkable pioneer woman's life as well as an unforgettable tale of our own American past."
228,Time For Kids: Butterflies!,Juvenile Nonfiction,"Butterflies There are 20,000 different kinds of butterflies in the world. Many have brightly colored wings. But butterflies don't start life as beauties. Every butterfly begins as a tiny egg that hatches into a caterpillar. Then an amazing change takes place. The caterpillar turns into a fluttering butterfly!"
267,The Annotated Charlotte's Web,Juvenile Nonfiction,"Charlotte's Web, one of America's best-loved children's books, is seamless in plot, poignant in its delineation of characters who will live in children's literature forever, and written with a seeming ease and fluency that make the reading as engrossing as it is effortless. The Annotated Charlotte's Web opens the door to the creator's workshop and allows the reader to glance over the shoulder of one of the most exacting and fastidious American writers, watching as he forms his masterpiece."
434,My Little House Crafts Book,Juvenile Nonfiction,"Make the same pioneer crafts that Laura did! In her beloved Little House books, Laura Ingalls Wilder tells of her pioneer childhood growing up on the frontier. Because the Ingallses often lived far away from any stores, and because they had little money, Laura and her family had to make most of their own household items, using whatever materials were available. Now, with MY LITTLE HOUSE CRAFTS BOOK, you can share a part of Laura's pioneer days by learning how to make the same things that she and her family made for their little homes on the frontier. Inside this book are easy, step-by-step instructions for 18 crafts taken straight from the pages of Laura's Little House books. All you need to get started are a few simple supplies, and you're on your way to making fun and useful pioneer crafts that you'll cherish forever."


In [7]:

pd.set_option("display.max_colwidth", 50)
category_mapping = {
    'Fiction': "Fiction",
    'Juvenile Fiction': "Children's Fiction",
    'Biography & Autobiography' : "Nonfiction",
    'History': "Nonfiction",
    'Literary Criticism': "Nonfiction",
    'Philosophy': "Nonfiction",
    'Religion': "Nonfiction", 
    'Comics & Graphic Novels': "Fiction",
    'Drama': "Fiction",
    'Juvenile nonfiction': "Children's Nonfiction",
    'Science': "Nonfiction",
    'Poetry': "Fiction"
}
books['simple_categories'] = books['categories'].map(category_mapping)


In [8]:
(~books['simple_categories'].isna()).sum() # in total with that simple mapping we could put ~3700 of the books in simple categories

np.int64(3686)

# Using huggingface to perform zero-shot classification

In [ ]:
from transformers import pipeline

# Use a tiny model for a quick check
# SMALL_MODEL = "finiteautomata/bertweet-base-sentiment-analysis" 
LARGE_MODEL = "facebook/bart-large-mnli"
fiction_categories = ["Fiction", "Nonfiction"]

pipe = pipeline("zero-shot-classification", 
                model=LARGE_MODEL,
                device=0,
                local_files_only=False) # Force redownload

print("Pipeline initialized successfully!")

result = pipe("This is a simple test sentence.", fiction_categories)
print(result)

Device set to use cpu


Pipeline initialized successfully!
{'sequence': 'This is a simple test sentence.', 'labels': ['Nonfiction', 'Fiction'], 'scores': [0.7708202004432678, 0.2291797697544098]}


In [27]:
# Lets check how the classifier is working 
seq = books.loc[books['simple_categories']== "Fiction", "description"].reset_index(drop=True)[0]
print(pipe(seq, fiction_categories)['labels'], pipe(seq, fiction_categories)['scores'])

seq = books.loc[books['simple_categories']== "Nonfiction", "description"].reset_index(drop=True)[0]
print(pipe(seq, fiction_categories)['labels'], pipe(seq, fiction_categories)['scores'])


['Fiction', 'Nonfiction'] [0.8438267111778259, 0.15617330372333527]
['Nonfiction', 'Fiction'] [0.875166654586792, 0.12483339011669159]


In [31]:
seq = books.loc[books['simple_categories']== "Nonfiction", "description"].reset_index(drop=True)[0]
result = pipe(seq, fiction_categories)
max_index = np.argmax(result['scores'])
category = result['labels'][max_index]
print(category)

Nonfiction


In [33]:
def generate_predictions(sequence, categories):

    predictions = pipe(sequence, categories)
    max_index = np.argmax(predictions['scores'])
    max_label = predictions['labels'][max_index]

    return max_label
